In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ncaaw-march-mania-2021/WDataFiles_Stage1/Conferences.csv
/kaggle/input/ncaaw-march-mania-2021/WDataFiles_Stage1/WNCAATourneyDetailedResults.csv
/kaggle/input/ncaaw-march-mania-2021/WDataFiles_Stage1/WSampleSubmissionStage1.csv
/kaggle/input/ncaaw-march-mania-2021/WDataFiles_Stage1/WRegularSeasonCompactResults.csv
/kaggle/input/ncaaw-march-mania-2021/WDataFiles_Stage1/WGameCities.csv
/kaggle/input/ncaaw-march-mania-2021/WDataFiles_Stage1/WNCAATourneySlots.csv
/kaggle/input/ncaaw-march-mania-2021/WDataFiles_Stage1/Cities.csv
/kaggle/input/ncaaw-march-mania-2021/WDataFiles_Stage1/WTeams.csv
/kaggle/input/ncaaw-march-mania-2021/WDataFiles_Stage1/WNCAATourneyCompactResults.csv
/kaggle/input/ncaaw-march-mania-2021/WDataFiles_Stage1/WSeasons.csv
/kaggle/input/ncaaw-march-mania-2021/WDataFiles_Stage1/WNCAATourneySeeds.csv
/kaggle/input/ncaaw-march-mania-2021/WDataFiles_Stage1/WRegularSeasonDetailedResults.csv
/kaggle/input/ncaaw-march-mania-2021/WDataFiles_Stage1/WTeamSpellings.c

In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

seeds = pd.read_csv('../input/ncaaw-march-mania-2021/WDataFiles_Stage2/WNCAATourneySeeds.csv')
tourney_results = pd.read_csv('../input/ncaaw-march-mania-2021/WDataFiles_Stage2/WNCAATourneyCompactResults.csv')
regular_results = pd.read_csv('../input/ncaaw-march-mania-2021/WDataFiles_Stage2/WRegularSeasonCompactResults.csv')
test_df = pd.read_csv('../input/ncaaw-march-mania-2021/WDataFiles_Stage2/WSampleSubmissionStage2.csv')

In [3]:
test_df['Season'] = test_df['ID'].apply(lambda x: int(x.split('_')[0]))
test_df['T1_TeamID'] = test_df['ID'].apply(lambda x: int(x.split('_')[1]))
test_df['T2_TeamID'] = test_df['ID'].apply(lambda x: int(x.split('_')[2]))

In [4]:
df_test = pd.merge(
    test_df,
    tourney_results,
    how='left',
    left_on=['Season', 'T1_TeamID'],
    right_on=['Season', 'WTeamID']).drop(columns='ID', axis=1)

In [5]:
df_test = pd.merge(
    test_df,
    tourney_results,
    how='left',
    left_on=['Season', 'T2_TeamID'],
    right_on=['Season', 'WTeamID']).drop(columns='ID', axis=1)

In [6]:
def prepare_data(df):
    dfswap = df[['Season', 'DayNum', 'LTeamID', 'LScore', 'WTeamID', 'WScore', 'WLoc', 'NumOT']]

    dfswap.loc[df['WLoc'] == 'H', 'WLoc'] = 'A'
    dfswap.loc[df['WLoc'] == 'A', 'WLoc'] = 'H'
    df.columns.values[6] = 'location'
    dfswap.columns.values[6] = 'location'         
    df.columns = [x.replace('W','T1_').replace('L','T2_') for x in list(df.columns)]
    dfswap.columns = [x.replace('L','T1_').replace('W','T2_') for x in list(dfswap.columns)]
    output = pd.concat([df, dfswap]).sort_index().reset_index(drop=True)
    
    return output

In [7]:
tourney_results = prepare_data(tourney_results)
regular_results = prepare_data(regular_results)

In [8]:
# convert to str, so the model would treat TeamID them as factors
regular_results['T1_TeamID'] = regular_results['T1_TeamID'].astype(str)
regular_results['T2_TeamID'] = regular_results['T2_TeamID'].astype(str)

# make it a binary task
regular_results['win'] = np.where(regular_results['T1_Score']>regular_results['T2_Score'], 1, 0)

def team_quality(season):
    """
    Calculate team quality for each season seperately. 
    Team strength changes from season to season (students playing change!)
    So pooling everything would be bad approach!
    """
    formula = 'win~-1+T1_TeamID+T2_TeamID'
    glm = sm.GLM.from_formula(formula=formula, 
                              data=regular_results.loc[regular_results.Season==season,:], 
                              family=sm.families.Binomial()).fit()
    
    # extracting parameters from glm
    quality = pd.DataFrame(glm.params).reset_index()
    quality.columns = ['TeamID','beta']
    quality['Season'] = season
    # taking exp due to binomial model being used
    quality['quality'] = np.exp(quality['beta'])
    # only interested in glm parameters with T1_, as T2_ should be mirroring T1_ ones
    quality = quality.loc[quality.TeamID.str.contains('T1_')].reset_index(drop=True)
    quality['TeamID'] = quality['TeamID'].apply(lambda x: x[10:14]).astype(int)
    return quality

In [9]:
team_quality = pd.concat([team_quality(2010),
                          team_quality(2011),
                          team_quality(2012),
                          team_quality(2013),
                          team_quality(2014),
                          team_quality(2015),
                          team_quality(2016),
                          team_quality(2017),
                          team_quality(2018),
                         team_quality(2019),
                         team_quality(2020),
                          team_quality(2021)]).reset_index(drop=True)

/opt/conda/lib/python3.7/site-packages/statsmodels/genmod/families/family.py:894: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
/opt/conda/lib/python3.7/site-packages/statsmodels/genmod/families/family.py:894: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
/opt/conda/lib/python3.7/site-packages/statsmodels/genmod/families/family.py:894: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))


In [10]:
 team_quality_T1 = team_quality[['TeamID','Season','quality']]
team_quality_T1.columns = ['T1_TeamID','Season','T1_quality']
team_quality_T2 = team_quality[['TeamID','Season','quality']]
team_quality_T2.columns = ['T2_TeamID','Season','T2_quality']

tourney_results['T1_TeamID'] = tourney_results['T1_TeamID'].astype(int)
tourney_results['T2_TeamID'] = tourney_results['T2_TeamID'].astype(int)
tourney_results = tourney_results.merge(team_quality_T1, on = ['T1_TeamID','Season'], how = 'left')
tourney_results = tourney_results.merge(team_quality_T2, on = ['T2_TeamID','Season'], how = 'left')

In [11]:
# we only have tourney results since year 2010
tourney_results = tourney_results.loc[tourney_results['Season'] >= 2010].reset_index(drop=True)

# not interested in pre-selection matches
tourney_results = tourney_results.loc[tourney_results['DayNum'] >= 136].reset_index(drop=True)

In [12]:
seeds['seed'] = seeds['Seed'].apply(lambda x: int(x[1:3]))
seeds['division'] = seeds['Seed'].apply(lambda x: x[0])

seeds_T1 = seeds[['Season','TeamID','seed','division']].copy()
seeds_T2 = seeds[['Season','TeamID','seed','division']].copy()
seeds_T1.columns = ['Season','T1_TeamID','T1_seed','T1_division']
seeds_T2.columns = ['Season','T2_TeamID','T2_seed','T2_division']

tourney_results = tourney_results.merge(seeds_T1, on = ['Season', 'T1_TeamID'], how = 'left')
tourney_results = tourney_results.merge(seeds_T2, on = ['Season', 'T2_TeamID'], how = 'left')

In [13]:
tourney_results['T1_powerrank'] = tourney_results.groupby(['Season','T1_division'])['T1_quality'].rank(method='dense', ascending=False).astype(int)
tourney_results['T2_powerrank'] = tourney_results.groupby(['Season','T2_division'])['T2_quality'].rank(method='dense', ascending=False).astype(int)

In [14]:
tourney_results['win'] = np.where(tourney_results['T1_Score'] > tourney_results['T2_Score'], 1, 0)

In [15]:
mean_win_ratio = pd.DataFrame({'seed_win_ratio': tourney_results.groupby('T1_seed')['win'].mean(),
                               'powerrank_win_ratio': tourney_results.groupby('T1_powerrank')['win'].mean()})
mean_win_ratio

,seed_win_ratio,powerrank_win_ratio
1,0.838542,0.839378
2,0.740000,0.729167
3,0.636364,0.652174
4,0.646018,0.629630
5,0.555556,0.565217
6,0.480519,0.459459
7,0.480519,0.473684
8,0.333333,0.428571
9,0.333333,0.298246
10,0.298246,0.310345


In [16]:
test_df['Season'] = test_df['ID'].apply(lambda x: int(x.split('_')[0]))
test_df['T1_TeamID'] = test_df['ID'].apply(lambda x: int(x.split('_')[1]))
test_df['T2_TeamID'] = test_df['ID'].apply(lambda x: int(x.split('_')[2]))

In [17]:
test_df = pd.merge(test_df, team_quality_T1, left_on=['Season', 'T1_TeamID'], right_on=['Season', 'T1_TeamID'], how='left')
test_df = pd.merge(test_df, team_quality_T2, left_on=['Season', 'T2_TeamID'], right_on=['Season', 'T2_TeamID'], how='left')

In [18]:
test_df = pd.merge(test_df,seeds_T1, left_on=['Season', 'T1_TeamID'], right_on=['Season', 'T1_TeamID'], how='left')
test_df.rename(columns={'Seed':'Seed1'}, inplace=True)
test_df = test_df.drop('T1_division', axis=1)

test_df = pd.merge(test_df,seeds_T2, left_on=['Season', 'T2_TeamID'], right_on=['Season', 'T2_TeamID'], how='left')
test_df.rename(columns={'Seed':'Seed2'}, inplace=True)
test_df = test_df.drop('T2_division', axis=1)

In [19]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression

In [20]:
def get_train_test(df, test_season):
    train_df = df.loc[df['Season']<test_season, cols+['win']]
    test_df = df.loc[df['Season']==test_season, cols+['win']]
    return train_df, test_df

In [21]:
tourney_results.tail()

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location,NumOT,T1_quality,T2_quality,T1_seed,T1_division,T2_seed,T2_division,T1_powerrank,T2_powerrank,win
1255,2019,151,3124,72,3332,67,N,0,1155.901977,235.078186,1,Y,2,Z,1,2,1
1256,2019,151,3163,76,3323,81,N,0,564.371758,488.623628,2,W,1,X,1,1,0
1257,2019,151,3323,81,3163,76,N,0,488.623628,564.371758,1,X,2,W,1,1,1
1258,2019,153,3124,82,3323,81,N,0,1155.901977,488.623628,1,Y,1,X,1,1,1
1259,2019,153,3323,81,3124,82,N,0,488.623628,1155.901977,1,X,1,Y,1,1,0


In [22]:
cols = ['T1_quality','T2_quality','T1_seed','T2_seed']

In [23]:
model1 = LogisticRegression()

In [24]:
#model 1
gloss = 0
seasons = [2015, 2016, 2017, 2018, 2019]

for season in seasons:
    train, test = get_train_test(tourney_results, season)
    model1.fit(train.drop('win', axis=1), train['win'])
    pred = model1.predict_proba(test.drop('win', axis=1))[:,1]
    loss = log_loss(test['win'], pred)
    print(season, loss)
    gloss += loss

print('average', gloss/len(seasons))

2015 0.6931471805599454
2016 0.6931471805599454
2017 0.6931471805599454
2018 0.6931471805599454
2019 0.6931471805599454
average 0.6931471805599454


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGIS

In [25]:
#model2=LGBMClassifier(num_leaves=200,min_child_weight=0.1,bagging_fraction=0.418,random_state= 47,reg_lambda=0.91,metric='logloss',learning_rate=0.141,min_data_in_leaf=96,bagging_seed=12,max_depth= -1,)
model2=LGBMClassifier(num_leaves=70,min_child_weight=0.1,bagging_fraction=0.418,random_state= 47,reg_lambda=0.91,metric='logloss',learning_rate=0.141,min_data_in_leaf=120,bagging_seed=12,max_depth=-1 ,verbosity= -1)

In [26]:
#model 2
gloss = 0
seasons = [2015, 2016, 2017, 2018, 2019]

for season in seasons:
    train, test = get_train_test(tourney_results, season)
    model2.fit(train.drop('win', axis=1), train['win'])
    pred = model2.predict_proba(test.drop('win', axis=1))[:,1]
    loss = log_loss(test['win'], pred)
    print(season, loss)
    gloss += loss

print('average', gloss/len(seasons))

[LightGBM] [Warning] min_data_in_leaf is set=120, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=120
[LightGBM] [Warning] bagging_fraction is set=0.418, subsample=1.0 will be ignored. Current value: bagging_fraction=0.418
2015 0.43880493817337674
[LightGBM] [Warning] min_data_in_leaf is set=120, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=120
[LightGBM] [Warning] bagging_fraction is set=0.418, subsample=1.0 will be ignored. Current value: bagging_fraction=0.418
2016 0.5236959336223583
[LightGBM] [Warning] min_data_in_leaf is set=120, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=120
[LightGBM] [Warning] bagging_fraction is set=0.418, subsample=1.0 will be ignored. Current value: bagging_fraction=0.418
2017 0.45646324977017927
[LightGBM] [Warning] min_data_in_leaf is set=120, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=120
[LightGBM] [Warning] bagging_fraction is set=0.418, subsample=1.0 w

In [27]:
model3=XGBClassifier(gamma=0.25,
                   n_estimators=5000,max_depth=3,colsample_bytree=0.8,                 
                   learning_rate=0.05,
      nthread = 12)

In [28]:
#model 3
gloss = 0
seasons = [2015, 2016, 2017, 2018, 2019]

for season in seasons:
    train, test = get_train_test(tourney_results, season)
    model3.fit(train.drop('win', axis=1), train['win'])
    pred = model3.predict_proba(test.drop('win', axis=1))[:,1]
    loss = log_loss(test['win'], pred)
    print(season, loss)
    gloss += loss

print('average', gloss/len(seasons))

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:29:32] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
2015 0.40798284875511726
[20:29:41] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


2016 0.5796162577785181
[20:29:51] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


2017 0.542979838109796
[20:30:02] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


2018 0.49717416681440746
[20:30:13] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


2019 0.4525112809778731
average 0.4960528784871423


In [29]:
model4=CatBoostClassifier(iterations=500,depth=10,colsample_bylevel=0.5,early_stopping_rounds=400,l2_leaf_reg=25,random_seed=42)

In [30]:
#model 4
gloss = 0
seasons = [2015, 2016, 2017, 2018, 2019]

for season in seasons:
    train, test = get_train_test(tourney_results, season)
    model4.fit(train.drop('win', axis=1), train['win'])
    pred = model4.predict_proba(test.drop('win', axis=1))[:,1]
    loss = log_loss(test['win'], pred)
    print(season, loss)
    gloss += loss

print('average', gloss/len(seasons))

0:	learn: 0.6870599	total: 51.9ms	remaining: 25.9s
1:	learn: 0.6829801	total: 52.7ms	remaining: 13.1s
2:	learn: 0.6794580	total: 53.4ms	remaining: 8.85s
3:	learn: 0.6745227	total: 53.9ms	remaining: 6.68s
4:	learn: 0.6621526	total: 56ms	remaining: 5.54s
5:	learn: 0.6514266	total: 67.5ms	remaining: 5.55s
6:	learn: 0.6412075	total: 77.2ms	remaining: 5.43s
7:	learn: 0.6323827	total: 83.7ms	remaining: 5.14s
8:	learn: 0.6231592	total: 85.6ms	remaining: 4.67s
9:	learn: 0.6144265	total: 88.3ms	remaining: 4.32s
10:	learn: 0.6075038	total: 89.2ms	remaining: 3.96s
11:	learn: 0.6001604	total: 94.8ms	remaining: 3.85s
12:	learn: 0.5928610	total: 101ms	remaining: 3.77s
13:	learn: 0.5852097	total: 104ms	remaining: 3.62s
14:	learn: 0.5777656	total: 110ms	remaining: 3.54s
15:	learn: 0.5713408	total: 113ms	remaining: 3.41s
16:	learn: 0.5658244	total: 114ms	remaining: 3.23s
17:	learn: 0.5605648	total: 115ms	remaining: 3.07s
18:	learn: 0.5550249	total: 118ms	remaining: 2.99s
19:	learn: 0.5527687	total: 119

In [31]:
modele1=LGBMClassifier(num_leaves=70,min_child_weight=0.1,bagging_fraction=0.418,random_state= 47,reg_lambda=0.91,metric='logloss',learning_rate=0.15,min_data_in_leaf=120,bagging_seed=12,max_depth= -1,verbosity= -1)

In [32]:
modele2=CatBoostClassifier(iterations=500,depth=10,colsample_bylevel=0.5,early_stopping_rounds=400,l2_leaf_reg=25,random_seed=42)

In [33]:
weights = {
    'modele1': 7, 
    
    'modele2': 13
}

In [34]:
#2019 0.40803928573534987
#average 0.4369542920444447
2019 0.4030772816385251
average 0.43556831467824664
2019 0.4008309409804568
average 0.4354967753388566
2019 0.3997640055905858
average 0.4356187043957222

SyntaxError: invalid syntax (<ipython-input-34-4e153778ba86>, line 3)

In [35]:
#ensembling model 
gloss = 0
for season in seasons:
    train, test = get_train_test(tourney_results, season)
    modele1.fit(train.drop('win', axis=1), train['win'])
    modele2.fit(train.drop('win', axis=1), train['win'])
    pred = ((modele1.predict_proba(test.drop('win', axis=1))[:,1]*weights['modele1']+modele2.predict_proba(test.drop('win', axis=1))[:,1]*weights['modele2'])/(weights['modele1']+weights['modele2']))
    loss = log_loss(test['win'], pred)
    #f1_score=accuracy_score(test['target'], pred)
    print(season, loss)
    gloss += loss

print('average', gloss/len(seasons))

[LightGBM] [Warning] min_data_in_leaf is set=120, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=120
[LightGBM] [Warning] bagging_fraction is set=0.418, subsample=1.0 will be ignored. Current value: bagging_fraction=0.418
0:	learn: 0.6870599	total: 640us	remaining: 319ms
1:	learn: 0.6829801	total: 1.06ms	remaining: 263ms
2:	learn: 0.6794580	total: 1.49ms	remaining: 247ms
3:	learn: 0.6745227	total: 1.83ms	remaining: 227ms
4:	learn: 0.6621526	total: 3.23ms	remaining: 320ms
5:	learn: 0.6514266	total: 9.88ms	remaining: 814ms
6:	learn: 0.6412075	total: 15.3ms	remaining: 1.08s
7:	learn: 0.6323827	total: 21.2ms	remaining: 1.3s
8:	learn: 0.6231592	total: 23.4ms	remaining: 1.27s
9:	learn: 0.6144265	total: 25.2ms	remaining: 1.23s
10:	learn: 0.6075038	total: 25.9ms	remaining: 1.15s
11:	learn: 0.6001604	total: 30.8ms	remaining: 1.25s
12:	learn: 0.5928610	total: 35.7ms	remaining: 1.34s
13:	learn: 0.5852097	total: 39.3ms	remaining: 1.36s
14:	learn: 0.5777656	total: 45.1ms	rema

In [36]:
submission_df = pd.read_csv('../input/ncaaw-march-mania-2021/WDataFiles_Stage2/WSampleSubmissionStage2.csv')
modele1.fit(tourney_results[cols], tourney_results['win'])
modele2.fit(tourney_results[cols], tourney_results['win'])
pred = ((modele1.predict_proba(test_df[cols])[:,1]*weights['modele1']+modele2.predict_proba(test_df[cols])[:,1]*weights['modele2'])/(weights['modele1']+weights['modele2']))
test_df['Pred'] = pred.clip(0, 1)
test_df.to_csv('submission.csv', columns=['ID','Pred'], index=None)
test_df

[LightGBM] [Warning] min_data_in_leaf is set=120, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=120
[LightGBM] [Warning] bagging_fraction is set=0.418, subsample=1.0 will be ignored. Current value: bagging_fraction=0.418
0:	learn: 0.6871056	total: 951us	remaining: 475ms
1:	learn: 0.6839465	total: 2.15ms	remaining: 536ms
2:	learn: 0.6810416	total: 3.38ms	remaining: 559ms
3:	learn: 0.6694318	total: 4.85ms	remaining: 602ms
4:	learn: 0.6557109	total: 11.8ms	remaining: 1.17s
5:	learn: 0.6435906	total: 15.4ms	remaining: 1.27s
6:	learn: 0.6328998	total: 17ms	remaining: 1.2s
7:	learn: 0.6229169	total: 23.1ms	remaining: 1.42s
8:	learn: 0.6128254	total: 28.6ms	remaining: 1.56s
9:	learn: 0.6050256	total: 29.6ms	remaining: 1.45s
10:	learn: 0.5949418	total: 32.9ms	remaining: 1.46s
11:	learn: 0.5861642	total: 38.4ms	remaining: 1.56s
12:	learn: 0.5785175	total: 44.4ms	remaining: 1.66s
13:	learn: 0.5694188	total: 48.4ms	remaining: 1.68s
14:	learn: 0.5621410	total: 54ms	remainin

,ID,Pred,Season,T1_TeamID,T2_TeamID,T1_quality,T2_quality,T1_seed,T2_seed
0,2021_3104_3112,0.382699,2021,3104,3112,619.453761,1281.751208,7,3
1,2021_3104_3116,0.349907,2021,3104,3116,619.453761,1471.832060,7,4
2,2021_3104_3124,0.259476,2021,3104,3124,619.453761,3370.327895,7,2
3,2021_3104_3125,0.910812,2021,3104,3125,619.453761,30.976567,7,12
4,2021_3104_3133,0.820391,2021,3104,3133,619.453761,50.539045,7,11
...,...,...,...,...,...,...,...,...,...
2011,2021_3450_3460,0.870313,2021,3450,3460,354.713228,29.217894,9,13
2012,2021_3450_3461,0.876360,2021,3450,3461,354.713228,22.318230,9,14
2013,2021_3452_3460,0.982695,2021,3452,3460,872.078824,29.217894,4,13
2014,2021_3452_3461,0.985138,2021,3452,3461,872.078824,22.318230,4,14
